In [1]:
import torch
from torch.utils.data import DataLoader as TorchDataLoader

from DL_vs_HateSpeech.models.model_v0 import ModelV0
from DL_vs_HateSpeech.loading_data.dataloader import DataLoader
from DL_vs_HateSpeech.training.training import (
    collate_fn,
    get_optimizer_and_criterion,
    train_epoch
)
from DL_vs_HateSpeech.evaluation.evaluate import evaluate

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
BATCH_SIZE = 32
LR = 1e-5
EPOCHS = 10

# Load Data
train_dataset = DataLoader(type="train")
val_dataset = DataLoader(type="val")

train_loader = TorchDataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = TorchDataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Initialize Model, Optimizer, Loss
model = ModelV0(clip_model_type="32").to(device)
optimizer, criterion = get_optimizer_and_criterion(model, lr=LR)

# Training loop
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    # Train
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.4f}")

    # Evaluate
    val_accuracy = evaluate(model, val_loader, device)
    print(f"Val Accuracy: {val_accuracy * 100:.2f}%")


/home/jack/anaconda3/envs/ee559/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Epoch 1/10


Training: 100%|██████████| 14/14 [01:04<00:00,  4.61s/it]


Train Loss: 0.6847


Evaluating: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


...

Val Accuracy: 57.05%

Epoch 10/10


Training: 100%|██████████| 14/14 [01:07<00:00,  4.85s/it]


Train Loss: 0.0232


Evaluating: 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]

Val Accuracy: 59.73%
